# 1.背景

在进行变量筛选后，通常我们需要：
1. 根据临床专家的意见加入一些重要变量，如受试者年龄、性别；
2. 处理多分类变量问题（通常是如果多分类变量对应的某一虚拟变量被纳入模型，则所有该多分类变量的所有虚拟变量都纳入模型中）；
3. 采用存在限制条件策略筛选得到的变量；

在处理以上三种情况后，重新对train model进行重新拟合。

下面我们以采用存在限制条件策略筛选变量后重新拟合模型为例。

# 2.Simulation（模拟实验）

## 2.1 导入数据和筛选变量

In [1]:
# Install required packages and library them
packages <- c("survival",
              "survMisc",
              "plyr",
              "dplyr",
              "glmnet")

for (i in packages) {
    if (!suppressMessages(require(i, character.only = TRUE, quietly = TRUE))) {
        install.packages(i, quietly = TRUE)
    }
}

In [2]:
load("model_train_workspace.Rdata")

## 2.2 查看已筛选得到的变量

In [3]:
train_model_lasso

[1] "Surv(AKI_time, AKI_status) ~ age + age_square + male + male_CKD_stage_G3a + age_BMI_TC"

此前经Lasso回归生成的新模型包括的变量有age，age_square，male，male_CKD_stage_G3a，age_BMI_TC。

## 2.3 采用Cox回归重新拟合模型

In [4]:
train_model_lasso_refit <- coxph(as.formula(train_model_lasso), data = dataset)

In [5]:
summary(train_model_lasso_refit)

Call:
coxph(formula = as.formula(train_model_lasso), data = dataset)

  n= 5000, number of events= 2700 

                        coef exp(coef)  se(coef)     z Pr(>|z|)    
age                1.048e-02 1.011e+00 7.047e-02 0.149  0.88177    
age_square         5.048e-05 1.000e+00 5.655e-04 0.089  0.92887    
male               3.674e-01 1.444e+00 4.767e-02 7.706  1.3e-14 ***
male_CKD_stage_G3a 9.572e-02 1.100e+00 5.149e-02 1.859  0.06304 .  
age_BMI_TC         2.604e-05 1.000e+00 9.656e-06 2.697  0.00699 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

                   exp(coef) exp(-coef) lower .95 upper .95
age                    1.011     0.9896    0.8802     1.160
age_square             1.000     0.9999    0.9989     1.001
male                   1.444     0.6926    1.3151     1.585
male_CKD_stage_G3a     1.100     0.9087    0.9948     1.217
age_BMI_TC             1.000     1.0000    1.0000     1.000

Concordance= 0.568  (se = 0.006 )
Likelihood ratio test= 

In [6]:
formula(train_model_lasso_refit)

Surv(AKI_time, AKI_status) ~ age + age_square + male + male_CKD_stage_G3a + 
    age_BMI_TC

采用COX回归重新拟合模型后新模型的预测变量包括age，age_square，male，male_CKD_stage_G3a，age_BMI_TC。

In [7]:
save.image("model_refit_workspace.Rdata")